<a href="https://colab.research.google.com/github/rklepov/hse-cs-ml-2018-2019/blob/homework/08-spark/HW/hw3/spark_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip search spark | grep INSTALLED || pip install pyspark==2.4.0 findspark

  INSTALLED: 2.4.0


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre/"

import findspark
findspark.init('/usr/local/lib/python3.6/dist-packages/pyspark/')

import pyspark

In [0]:
from pyspark.sql import types, functions

from pyspark.ml import classification, pipeline, feature, evaluation

In [0]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Загрузка данных

Будем использовать датасет [Adult](https://archive.ics.uci.edu/ml/datasets/Adult "UCI Machine Learning Repository: Adult Data Set") (потому что он на букву **A**).

In [5]:
%%bash

test -f adult.data || wget -q -O adult.data 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
test -f adult.test || wget -q -O adult.test 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'

ls -hl adult.*

-rw-r--r-- 1 root root 3.8M Aug 10  1996 adult.data
-rw-r--r-- 1 root root 2.0M Aug 10  1996 adult.test


### Схема данных

In [0]:
adult_cols = [
    ('age', 'integer'),
    ('workclass', 'string'),
    ('fnlwgt', 'float'),
    ('education', 'string'),
    ('education-num', 'float'),
    ('marital-status', 'string'),
    ('occupation', 'string'),
    ('relationship', 'string'),
    ('race', 'string'),
    ('sex', 'string'),
    ('capital-gain', 'float'),
    ('capital-loss', 'float'),
    ('hours-per-week', 'float'),
    ('native-country', 'string'),
    ('income', 'string'),
]

In [0]:
import functools

adult_schema = functools.reduce(lambda x,y: x.add(*y), adult_cols, types.StructType())

In [8]:
adult_train_df = (
    spark
    .read
    .option('header', 'false')
    .schema(adult_schema)
    .csv('adult.data')
)

adult_train_df.show(5)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education-num|     marital-status|        occupation|  relationship|  race|    sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|       

*В файле с тестовым датасетом первую строку нужно удалить. Также в тестовом датасете значение целевой переменной почему-то содержит точку `"."` в конце*.

In [9]:
adult_test_df = (
    spark
    .read
    .option('header', 'true')
    .schema(adult_schema)
    .csv('adult.test')
    .withColumn('income', functions.substring_index('income', '.', 1)) # "<=50K." -> '<=50K'
)

adult_test_df.show(5)

+---+----------+--------+-------------+-------------+-------------------+------------------+------------+------+-------+------------+------------+--------------+--------------+------+
|age| workclass|  fnlwgt|    education|education-num|     marital-status|        occupation|relationship|  race|    sex|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+----------+--------+-------------+-------------+-------------------+------------------+------------+------+-------+------------+------------+--------------+--------------+------+
| 25|   Private|226802.0|         11th|          7.0|      Never-married| Machine-op-inspct|   Own-child| Black|   Male|         0.0|         0.0|          40.0| United-States| <=50K|
| 38|   Private| 89814.0|      HS-grad|          9.0| Married-civ-spouse|   Farming-fishing|     Husband| White|   Male|         0.0|         0.0|          50.0| United-States| <=50K|
| 28| Local-gov|336951.0|   Assoc-acdm|         12.0| Married-civ-spouse|   Prot

### Категориальные признаки
(исключаем целевую переменную)

In [10]:
cat_cols = [ x[0] for x in adult_train_df.dtypes if 'string' == x[1] ][:-1]

cat_cols

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [11]:
adult_train_df.select(cat_cols).toPandas().describe().T

,count,unique,top,freq
workclass,32561,9,Private,22696
education,32561,16,HS-grad,10501
marital-status,32561,7,Married-civ-spouse,14976
occupation,32561,15,Prof-specialty,4140
relationship,32561,6,Husband,13193
race,32561,5,White,27816
sex,32561,2,Male,21790
native-country,32561,42,United-States,29170


### Числовые признаки

In [12]:
num_cols = [ x[0] for x in adult_train_df.dtypes if 'string' != x[1] ]

num_cols

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

In [13]:
adult_train_df.select(num_cols).toPandas().describe().T

,count,mean,std,min,25%,50%,75%,max
age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0
fnlwgt,32561.0,189777.859375,105549.703125,12285.0,117827.0,178356.0,237051.0,1484705.0
education-num,32561.0,10.080679,2.572562,1.0,9.0,10.0,12.0,16.0
capital-gain,32561.0,1077.649170,7385.911621,0.0,0.0,0.0,0.0,99999.0
capital-loss,32561.0,87.303833,403.014771,0.0,0.0,0.0,0.0,4356.0
hours-per-week,32561.0,40.437454,12.347933,1.0,40.0,40.0,45.0,99.0


## Предобработка данных

[One hot encoding](https://spark.apache.org/docs/latest/ml-features.html#onehotencoderestimator "Extracting, transforming and selecting features - Spark Documentation") для категориальных признаков.

In [0]:
pp_stages = []

for cat_col in cat_cols:
    string_indexer = feature.StringIndexer(inputCol=cat_col, outputCol=cat_col + 'Idx')
    ohe_encoder = feature.OneHotEncoderEstimator(inputCols=[string_indexer.getOutputCol()],
                                                 outputCols=[cat_col + 'OHE'])
    pp_stages += [string_indexer, ohe_encoder]
    
label_string_indexer = feature.StringIndexer(inputCol='income', outputCol='label')
pp_stages += [label_string_indexer]

[Стандартизация](https://spark.apache.org/docs/latest/ml-features.html#standardscaler "Extracting, transforming and selecting features - Spark Documentation") для числовых признаков.

In [0]:
num_col_assembler = feature.VectorAssembler(inputCols=num_cols, outputCol='num_features')
std_scaler = feature.StandardScaler(inputCol='num_features', outputCol='num_features_scaled')

pp_stages += [num_col_assembler, std_scaler]

### Подготовка признаков

In [0]:
assembler_inputs = [c + "OHE" for c in cat_cols] + ['num_features_scaled']
assembler = feature.VectorAssembler(inputCols=assembler_inputs, outputCol='features')

pp_stages += [assembler]

In [17]:
pp_pipeline = pipeline.Pipeline().setStages(pp_stages)

preprocessor = pp_pipeline.fit(adult_train_df)

train = (
    preprocessor
    .transform(adult_train_df)
    .select('features', 'label')
)

train.show(10)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(100,[4,10,24,32,...|  0.0|
|(100,[1,10,23,31,...|  0.0|
|(100,[0,8,25,38,4...|  0.0|
|(100,[0,13,23,38,...|  0.0|
|(100,[0,10,23,29,...|  0.0|
|(100,[0,11,23,31,...|  0.0|
|(100,[0,18,28,34,...|  0.0|
|(100,[1,8,23,31,4...|  1.0|
|(100,[0,11,24,29,...|  1.0|
|(100,[0,10,23,31,...|  1.0|
+--------------------+-----+
only showing top 10 rows



## Модель

Попробуем использовать [логистическую регрессию](https://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression "Classification and regression - Spark Documentation") (потому что для неё доступно [summary](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.classification.LogisticRegressionSummary "pyspark.ml package &#8212; PySpark master documentation")).

In [0]:
lr = classification.LogisticRegression(labelCol='label', featuresCol='features')

In [0]:
lr_model = lr.fit(train)

train_summary = lr_model.summary

In [0]:
train_evaluator = evaluation.BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')

pr_auc_train = train_evaluator.evaluate(train_summary.predictions, {train_evaluator.metricName: 'areaUnderPR'})

In [21]:
print(f'Train summary:\nAccuracy: {train_summary.accuracy:.3f}'
      f', ROC AUC: {train_summary.areaUnderROC:.3f}, PR AUC: {pr_auc_train:.3f}')

Train summary:
Accuracy: 0.853, ROC AUC: 0.909, PR AUC: 0.771


### Оценка модели.

In [22]:
test = (
    preprocessor
    .transform(adult_test_df)
    .select('features', 'label')
)

test.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(100,[0,13,24,35,...|  0.0|
|(100,[0,8,23,39,4...|  0.0|
|(100,[2,14,23,41,...|  1.0|
|(100,[0,9,23,35,4...|  1.0|
|(100,[3,9,24,36,4...|  0.0|
+--------------------+-----+
only showing top 5 rows



In [23]:
predictions = lr_model.transform(test)

predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(100,[0,13,24,35,...|  0.0|[6.05378303475958...|[0.99765654569464...|       0.0|
|(100,[0,8,23,39,4...|  0.0|[2.01085938710698...|[0.88193253742927...|       0.0|
|(100,[2,14,23,41,...|  1.0|[0.47709244026478...|[0.61706106222511...|       0.0|
|(100,[0,9,23,35,4...|  1.0|[-1.2073270571714...|[0.23017433905550...|       1.0|
|(100,[3,9,24,36,4...|  0.0|[6.65264144509033...|[0.99871105499591...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [0]:
test_evaluator = evaluation.BinaryClassificationEvaluator(rawPredictionCol='rawPrediction')

test_evaluator.evaluate(predictions)

pr_auc_test = test_evaluator.evaluate(predictions, {test_evaluator.metricName: 'areaUnderPR'})

In [0]:
test_summary = lr_model.evaluate(test)

In [26]:
print(f'Test summary:\nAccuracy: {test_summary.accuracy:.3f}'
      f', ROC AUC: {test_summary.areaUnderROC:.3f}, PR AUC: {pr_auc_test:.3f}')

Test summary:
Accuracy: 0.853, ROC AUC: 0.905, PR AUC: 0.763


## *References*

[1] [CSV Files &mdash; Databricks Documentation](https://docs.databricks.com/spark/latest/data-sources/read-csv.html "CSV Files &mdash; Databricks Documentation")

[2] [Binary Classification Example &mdash; Databricks Documentation](https://docs.databricks.com/spark/latest/mllib/binary-classification-mllib-pipelines.html "Binary Classification Example &mdash; Databricks Documentation")

[3] [Machine Learning with PySpark and MLlib — Solving a Binary Classification Problem](https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa "Machine Learning with PySpark and MLlib — Solving a Binary Classification Problem")

[4] [Apache Spark MLlib Tutorial — Part 1: Regression](https://towardsdatascience.com/apache-spark-mllib-tutorial-ec6f1cb336a9 "Apache Spark MLlib Tutorial — Part 1: Regression – Towards Data Science")

[5] [Apache Spark MLlib Tutorial — Part 2: Feature Transformation](https://towardsdatascience.com/apache-spark-mllib-tutorial-7aba8a1dce6e "Apache Spark MLlib Tutorial — Part 2: Feature Transformation – Towards Data Science")

[6] [Apache Spark MLlib Tutorial — Part 3: Complete Classification Workflow](https://towardsdatascience.com/apache-spark-mllib-tutorial-part-3-complete-classification-workflow-a1eb430ad069 "Apache Spark MLlib Tutorial — Part 3: Complete Classification Workflow – Towards Data Science")

[7] [Apache Spark Machine Learning Tutorial | MapR](https://mapr.com/blog/apache-spark-machine-learning-tutorial "Apache Spark Machine Learning Tutorial | MapR")